In [ ]:
# STL bounding box
# pip install numpy-stl
import numpy as np
from stl import mesh
import gzip
import os

def is_compressed(file_path):
    return file_path.endswith('.gz')

def decompress_stl_gz(gz_file_path, decompressed_file_path):
    with gzip.open(gz_file_path, 'rb') as f_in:
        with open(decompressed_file_path, 'wb') as f_out:
            f_out.write(f_in.read())

def get_bounding_box(stl_file_path):
    # Load the STL file
    your_mesh = mesh.Mesh.from_file(stl_file_path)

    # Get the minimum and maximum coordinates along each axis
    min_x, max_x = np.min(your_mesh.x), np.max(your_mesh.x)
    min_y, max_y = np.min(your_mesh.y), np.max(your_mesh.y)
    min_z, max_z = np.min(your_mesh.z), np.max(your_mesh.z)

    # Define the bounding box
    bounding_box = {
        'hullXmin': min_x, 'hullXmax': max_x,
        'hullYmin': min_y, 'hullYmax': 0,
        'hullZmin': min_z, 'hullZmax': max_z
    }

    return bounding_box

def write_bounding_box_to_file(bounding_box, file_path):
    with open(file_path, 'w') as file:
        for key, value in bounding_box.items():
            file.write(f"{key}  {value:.4f};\n")

# Example usage
file_path = 'hull.stl'
decompressed_file_path = 'decompressed_file.stl'
output_file_path = 'bounding_box.txt'

if is_compressed(file_path):
    # Decompress the STL file if it is compressed
    decompress_stl_gz(file_path, decompressed_file_path)
    stl_file_path = decompressed_file_path
else:
    stl_file_path = file_path

# Get the bounding box of the STL file
bounding_box = get_bounding_box(stl_file_path)

# Write the bounding box to a file
write_bounding_box_to_file(bounding_box, output_file_path)

# Remove the decompressed file if it was created
if is_compressed(file_path):
    os.remove(decompressed_file_path)

print(f"Bounding box data written to {output_file_path}")


In [ ]:
# calculate the volume of a stl geometry clipped by z-plane using python
# pip install numpy-stl
import numpy as np
from stl import mesh
import gzip
import os

def is_compressed(file_path):
    return file_path.endswith('.gz')

def decompress_stl_gz(gz_file_path, decompressed_file_path):
    with gzip.open(gz_file_path, 'rb') as f_in:
        with open(decompressed_file_path, 'wb') as f_out:
            f_out.write(f_in.read())

def get_volume(stl_file_path, z_clip):
    # Load the STL file
    your_mesh = mesh.Mesh.from_file(stl_file_path)

    # Clip the mesh by the z-plane
    clipped_mesh = your_mesh.clip([0, 0, -1], [0, 0, z_clip])  # Fixed the clipping method

    # Calculate the volume of the clipped mesh
    volume, _ = clipped_mesh.get_mass_properties()

    return volume

# Example usage
file_path = 'hull.stl'
decompressed_file_path = 'decompressed_file.stl'
z_clip = 0.22

if is_compressed(file_path):
    # Decompress the STL file if it is compressed
    decompress_stl_gz(file_path, decompressed_file_path)
    stl_file_path = decompressed_file_path
else:
    stl_file_path = file_path

# Get the volume of the STL file clipped by the z-plane
volume = get_volume(stl_file_path, z_clip)

# Remove the decompressed file if it was created
if is_compressed(file_path):
    os.remove(decompressed_file_path)

print(f"Volume of the geometry clipped by z={z_clip} is {volume:.4f} cubic units")


In [ ]:
import numpy as np
from stl import mesh

def triangle_volume(vertex1, vertex2, vertex3):
  # Calculate the volume of a tetrahedron based on its vertices
  return abs(np.dot(vertex1 - vertex3, np.cross(vertex2 - vertex3, vertex1 - vertex3))) / 6

def approximate_volume_below_z(mesh, z_clip):
  total_volume = 0
  for i in range(len(mesh.vectors)):
    triangle = mesh.vectors[i]
    # Check if the triangle is below the z_clip
    if np.all(triangle[:, 2] <= z_clip):
      # Calculate the volume of the triangle projected down to the z=0 plane
      total_volume += triangle_volume(triangle[0], triangle[1], triangle[2])
    # Additional logic can be added here to handle triangles intersecting the z_clip plane
  return total_volume

# Load the STL file
your_mesh = mesh.Mesh.from_file('hull.stl')

# Calculate the volume of the mesh below a specific z_clip
z_clip = 0.3
volume = approximate_volume_below_z(your_mesh, z_clip)

print(f"Approximate volume below z={z_clip}: {volume} cubic units")

In [18]:
from stl import mesh
import numpy as np

def calculate_volume(stl_path):
    # Load the STL file
    model_mesh = mesh.Mesh.from_file(stl_path)
    
    # Extract the vertex coordinates of each triangle
    vertices = model_mesh.vectors
    
    # Compute the volume using the divergence theorem
    volume = np.sum(
        vertices[:,0,:] * np.cross(vertices[:,1,:], vertices[:,2,:]), axis=1) / 6.0
    
    # The volume is the absolute value (in case the triangles are not consistently oriented)
    return np.abs(volume.sum())*1000

# Path to your STL file
stl_path = 'hull_clipped_para.stl'
print(f'The volume of the STL file is: {calculate_volume(stl_path)} cubic units')


The volume of the STL file is: 101.9168347120285 cubic units


In [14]:
import numpy as np
from stl import mesh

def clip_stl_by_z_plane(file_path, z_plane, output_path):
    # Load the STL file
    original_mesh = mesh.Mesh.from_file(file_path)
    
    # Find triangles below the specified z-plane
    below_plane = original_mesh.vectors[:,:,2] < z_plane
    
    # Filter to keep only the triangles completely below the z-plane
    keep_triangles = below_plane.all(axis=1)
    
    # Create a new mesh with these triangles
    new_mesh = mesh.Mesh(np.zeros(np.count_nonzero(keep_triangles), dtype=mesh.Mesh.dtype))
    new_mesh.vectors = original_mesh.vectors[keep_triangles]
    
    # Save the clipped mesh as a new STL file
    new_mesh.save(output_path)
    print(f"Clipped STL saved as {output_path}")

# Usage example
clip_stl_by_z_plane('hull.stl', z_plane=0.22, output_path='hull_clipped.stl')
# just removed the triangle that are above the z=0.22 plane, output is not solid

Clipped STL saved as hull_clipped.stl
